In [144]:
import numpy as np

## Perceptrons


In [136]:
class Perceptron:
    def __init__(self, input_size, lr=0.1, epochs=1000):
        self.weights = np.zeros(input_size + 1)
        self.lr = lr
        self.epochs = epochs

    def activate(self, z):
        return 1 if z > 0 else 0

    def ans(self, x):
        z = np.dot(x, self.weights[1:]) + self.weights[0]
        return self.activate(z)

    def train(self, X, y):
        for i in range(self.epochs):
            for x, target in zip(X, y):
                output = self.ans(x)
                error = (target - output) * self.lr
                self.weights[1:] += error * x
                self.weights[0] += error


In [137]:
X = np.array([[0, 0],
              [0, 1],
              [1, 0],
              [1, 1]])
y = np.array([0, 0, 0, 1])

# Train perceptron
p = Perceptron(input_size=2)
p.train(X, y)

# Test predictions
for input_vec in X:
    print(f"{input_vec} => {p.ans(input_vec)}")  # Use `ans` to get prediction


[0 0] => 0
[0 1] => 0
[1 0] => 0
[1 1] => 1


## Single Node

In [138]:
class Node:
    def __init__(self, input_size, lr=0.01, epochs=1000):
        self.weights = np.zeros(input_size)
        self.bias = 0
        self.lr = lr
        self.epochs = epochs

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

    def train(self, X, y):
        n = X.shape[0]

        for i in range(self.epochs):
            y_pred = self.predict(X)
            dw = (2/n) * np.dot(X.T, (y_pred - y))
            db = (2/n) * np.sum(y_pred - y)
            self.weights -= self.lr * dw
            self.bias    -= self.lr * db

In [139]:
X = np.array([1,2,3,4,5,6])  # Shape: (6, 1)
y = np.array([3, 6, 9, 12, 15, 18])           # y = 3x
input_size = int(input("Enter input size"))
model = Node(input_size=1, lr=0.01, epochs=1000)
X = X.reshape(-1, input_size)  # Reshape X
model.train(X, y)
for x_val in X:
    print(f"{x_val[0]} => {model.predict(x_val.reshape(1, -1))[0]:.2f}")

1 => 3.01
2 => 6.01
3 => 9.01
4 => 12.00
5 => 15.00
6 => 17.99


## Neural Network

In [ ]:
class Layer:
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(input_size, output_size)
        self.biases = np.zeros(output_size)
        self.last_output = None

    def forward(self, x):
        self.last_input = x
        self.last_output = np.dot(x, self.weights) + self.biases
        return self.relu(self.last_output)

    def backward(self, x, grad, lr):
        # gradient descent
        dw = np.dot(x.T, grad)  # Gradient w.r.t. weights
        db = np.sum(grad, axis=0)  # Gradient w.r.t. biases

        grad_clip = 10
        dw = np.clip(dw, -grad_clip, grad_clip)
        db = np.clip(db, -grad_clip, grad_clip)
        
        self.weights -= lr * dw
        self.biases -= lr * db
        return np.dot(grad, self.weights.T)  # Gradient for the next layer

    def relu(self, z):
        return np.maximum(0, z)  # ReLU activation

    def relu_derivative(self, z):
        return np.where(z > 0, 1, 0)  # Derivative of ReLU

In [147]:
class NeuralNetwork:
    def __init__(self, layer_sizes, lr=0.001):  
        # Create layers based on the given architecture
        self.lr = lr
        self.layers = [Layer(layer_sizes[i], layer_sizes[i+1]) for i in range(len(layer_sizes)-1)]

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backward(self, x, y_pred, y_true):
        loss_grad = 2 * (y_pred - y_true)  # MSE loss gradient
        for i in reversed(range(1, len(self.layers))):
            loss_grad = self.layers[i].backward(self.layers[i-1].last_output, loss_grad, self.lr)
        self.layers[0].backward(x, loss_grad, self.lr)

    def train(self, X, y, epochs=1000):
        for _ in range(epochs):
            for x_i, y_i in zip(X, y):
                x_input = x_i.reshape(1, -1)  # Ensure the input is a 2D array (single sample)
                for layer in self.layers:
                    x_input = layer.forward(x_input)
                    layer.last_output = x_input  # store for backward pass
                self.backward(x_i.reshape(1, -1), x_input, y_i.reshape(1, -1))  # Reshape for consistency

    def predict(self, x):
        x = x.reshape(1, -1)
        for layer in self.layers:
            x = layer.forward(x)
        return x

# Test the implementation on x^2 (squared function)
# Create a 3-layer network: input-1 → hidden-8 → output-1
nn = NeuralNetwork([1, 8, 1], lr=0.01)  # Using 1 input, 8 hidden neurons, and 1 output

# x^2 data
X = np.array([[0], [1], [2], [3], [4], [5]])  # Inputs
y = np.array([[0], [1], [4], [9], [16], [25]])  # Outputs (x^2)

# Train the model
nn.train(X, y, epochs=10000)  # Increase epochs for better learning

# Test predictions
for x in X:
    print(f"{x[0]} => {nn.predict(x)[0][0]:.2f}")


0 => 0.00
1 => 3.19
2 => 6.39
3 => 9.59
4 => 12.79
5 => 15.99


In [148]:
nn = NeuralNetwork([2, 4, 1], lr=0.01) 
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])  # AND gate inputs
y = np.array([[0], [0], [0], [1]])  # AND gate outputs
nn.train(X, y, epochs=10000)  
for x in X:
    print(f"{x} => {nn.predict(x)[0][0]:.2f}")

[0 0] => 0.00
[0 1] => 0.00
[1 0] => 0.00
[1 1] => 1.00
